In [1]:
import pandas as pd
import numpy as np

In [3]:
# read in the data
origin = pd.read_csv("originaldata.csv", low_memory=False)

In [4]:
list(origin)

['RA_Report #',
 'RA_CAERS Created Date',
 'AEC_Event Start Date',
 'PRI_Product Role',
 'PRI_Reported Brand/Product Name',
 'PRI_FDA Industry Code',
 'PRI_FDA Industry Name',
 'CI_Age at Adverse Event',
 'CI_Age Unit',
 'CI_Gender',
 'AEC_One Row Outcomes',
 'SYM_One Row Coded Symptoms']

In [6]:
origin[:5]

,RA_Report #,RA_CAERS Created Date,AEC_Event Start Date,PRI_Product Role,PRI_Reported Brand/Product Name,PRI_FDA Industry Code,PRI_FDA Industry Name,CI_Age at Adverse Event,CI_Age Unit,CI_Gender,AEC_One Row Outcomes,SYM_One Row Coded Symptoms
0,65325,1/1/04,8/4/03,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL..."
1,65333,1/1/04,NaN,Suspect,KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...,13,Ice Cream Prod,NaN,Not Available,Female,VISITED AN ER,"NAUSEA, DYSGEUSIA, DIARRHOEA"
2,65335,1/1/04,11/24/03,Suspect,ENFAMIL LIPIL BABY FORMULA,40,Baby Food Prod,3.0,Month(s),Not Available,NON-SERIOUS INJURIES/ ILLNESS,"GASTROINTESTINAL DISORDER, VOMITING"
3,65336,1/1/04,NaN,Suspect,ENFIMIL LIPIL BABY FORMULA,40,Baby Food Prod,NaN,Not Available,Not Available,VISITED A HEALTH CARE PROVIDER,"GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION"
4,65345,1/1/04,12/21/03,Suspect,"FRITO LAY FUNYUNS ONION FLAVOR, ONION RINGS",7,Snack Food Item,10.0,Year(s),Male,NON-SERIOUS INJURIES/ ILLNESS,CHOKING


In [5]:
# define variables types:
origin['RA_Report #'] = origin['RA_Report #'].astype('str')  
#origin['RA_CAERS Created Date'] = origin['RA_CAERS Created Date'].astype('str') 
#origin['AEC_Event Start Date'] 
origin['PRI_Product Role'] = origin['PRI_Product Role'].astype('str') 
origin['PRI_Reported Brand/Product Name'] = origin['PRI_Reported Brand/Product Name'].astype('str')
origin['PRI_FDA Industry Code'] = origin['PRI_FDA Industry Code'].astype('str')           
origin['PRI_FDA Industry Name'] = origin['PRI_FDA Industry Name'].astype('str')
origin['CI_Age Unit'] = origin['CI_Age Unit'].astype('str')
origin['CI_Gender'] = origin['CI_Gender'].astype('str')
origin['SYM_One Row Coded Symptoms'] = origin['SYM_One Row Coded Symptoms'].astype('str')

# convert last column from strings to lists (time consuming!)
# for i in range(50):
#     #print i
#     origin['SYM_One Row Coded Symptoms'][i] = origin['SYM_One Row Coded Symptoms'][i].split(',')
    

In [6]:
# missing data: not available, etc. 
origin['CI_Age Unit'] = origin['CI_Age Unit'].replace('Not Available', np.nan)
# Deal with decades
origin.ix[origin['CI_Age Unit'] == 'Decade(s)', 'CI_Age at Adverse Event'] *= 10
origin.ix[origin['CI_Age Unit'] == 'Decade(s)', 'CI_Age Unit'] = 'Year(s)'

origin['CI_Age Unit'].value_counts()


# for i in range(50):
#     if origin['CI_Age Unit'][i] == "Not Available":
#         origin['CI_Age Unit'][i] = np.nan
        
#origin.ix[['Not Available'],['CI_Age Unit']] = np.nan

Year(s)     46990
Month(s)     1153
Week(s)       162
Day(s)        138
Name: CI_Age Unit, dtype: int64

In [7]:
# remove outliers, and do descriptive analysis for each category of CI_Age Unit
# days
#origin[origin['CI_Age Unit'] == 'Day(s)'].describe()

for i in range(len(origin.index)):
    if (origin['CI_Age at Adverse Event'][i] >= 31) & (origin['CI_Age Unit'][i] == 'Day(s)'):
        origin['CI_Age at Adverse Event'][i] /= 30
        origin['CI_Age Unit'][i] = 'Month(s)'
        
days = origin[origin['CI_Age Unit'] == 'Day(s)']
#days
days['CI_Age at Adverse Event'].describe()


/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


count    86.000000
mean     10.465116
std       9.223624
min       0.000000
25%       2.250000
50%       7.500000
75%      18.750000
max      30.000000
Name: CI_Age at Adverse Event, dtype: float64

In [8]:
origin['CI_Age Unit'].value_counts()

Year(s)     46990
Month(s)     1205
Week(s)       162
Day(s)         86
Name: CI_Age Unit, dtype: int64

In [9]:
# weeks
weeks = origin[origin['CI_Age Unit'] == 'Week(s)']
weeks['CI_Age at Adverse Event'].describe()

for i in range(len(origin.index)):
    if (origin['CI_Age at Adverse Event'][i] > 4) & (origin['CI_Age Unit'][i] == 'Week(s)'):
        origin['CI_Age at Adverse Event'][i] /= 4
        origin['CI_Age Unit'][i] = 'Month(s)'
    if (origin['CI_Age at Adverse Event'][i] <= 4) & (origin['CI_Age Unit'][i] == 'Week(s)'):
        origin['CI_Age at Adverse Event'][i] *= 7
        origin['CI_Age Unit'][i] = 'Day(s)'
        
weeks = origin[origin['CI_Age Unit'] == 'Week(s)']
weeks
weeks['CI_Age at Adverse Event'].describe()


/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: CI_Age at Adverse Event, dtype: float64

In [10]:
# remove outliers, and do descriptive analysis for each category of CI_Age Unit
# month

for i in range(len(origin.index)):
    if (origin['CI_Age at Adverse Event'][i] > 12) & (origin['CI_Age Unit'][i] == 'Month(s)'):
        origin['CI_Age at Adverse Event'][i] /= 12
        origin['CI_Age Unit'][i] = 'Year(s)'
        
month = origin[origin['CI_Age Unit'] == 'Month(s)']
month
month['CI_Age at Adverse Event'].describe()

#origin[(origin['CI_Age at Adverse Event'] >= 36) & (origin['CI_Age Unit'] == 'Month(s)')] = np.nan
#months = origin[origin['CI_Age Unit'] == 'Month(s)']
#months['CI_Age at Adverse Event'].describe()

/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


count    998.000000
mean       5.237792
std        3.074188
min        0.000000
25%        2.500000
50%        5.000000
75%        8.000000
max       12.000000
Name: CI_Age at Adverse Event, dtype: float64

In [11]:
# remove outliers, and do descriptive analysis for each category of CI_Age Unit
# years: years >= 100 can be viewed as outliers
origin[(origin['CI_Age at Adverse Event']>= 100) & (origin['CI_Age Unit'] == 'Year(s)')] = np.nan
years = origin[origin['CI_Age Unit'] == 'Year(s)']
years['CI_Age at Adverse Event'].describe()


count    47309.000000
mean        51.514467
std         22.257302
min          1.000000
25%         36.000000
50%         54.000000
75%         69.000000
max         99.000000
Name: CI_Age at Adverse Event, dtype: float64

In [12]:
origin['CI_Age Unit'].value_counts()

Year(s)     47309
Month(s)      998
Day(s)        116
Name: CI_Age Unit, dtype: int64

In [13]:
# after 
46990 + 1153 + 162 + 138

48443

In [14]:
# after: the difference is outliers 
47309 + 998 + 116

48423

In [15]:
# create an age_interval based on age variable

origin['Age_interval'] = np.nan

yearsbin = [0, 20, 40, 60, 80, 100]
yearscategory = ['0-20', '21-40', '41-60','61-80','81-100']
year_interval = pd.cut(origin['CI_Age at Adverse Event'][origin['CI_Age Unit'] == 'Year(s)'], yearsbin, labels=yearscategory)

origin['Age_interval'][origin['CI_Age Unit'] == 'Year(s)'] = year_interval

#year_interval
#df.loc[df['B'] == 3, 'A'].
#origin.loc[origin['CI_Age Unit'] == 'Month(s)', 'Age_interval']

/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
# age_interval: month
monthsbin = [0, 3, 6, 9, 12]
monthscategory = ['0-3', '3-6', '6-9','9-12']
month_interval = pd.cut(origin['CI_Age at Adverse Event'][origin['CI_Age Unit'] == 'Month(s)'], monthsbin, labels=monthscategory)

origin['Age_interval'][origin['CI_Age Unit'] == 'Month(s)'] = month_interval

/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
# age_interval: days
daysbin = [0, 5, 10, 15, 20, 25, 30]
dayscategory = ['0-5', '5-10', '10-15','15-20', '20-25', '25-30']
days_interval = pd.cut(origin['CI_Age at Adverse Event'][origin['CI_Age Unit'] == 'Day(s)'], daysbin, labels=dayscategory)

origin['Age_interval'][origin['CI_Age Unit'] == 'Day(s)'] = days_interval

/Users/huilin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
#origin['Age_interval'].describe()

In [19]:
# Gender: missing data 
origin['CI_Gender'].value_counts()
origin['CI_Gender'] = origin['CI_Gender'].replace('Not Available', np.nan)
origin['CI_Gender'] = origin['CI_Gender'].replace('Unknown', np.nan)
origin['CI_Gender'] = origin['CI_Gender'].replace('Not Reported', np.nan)
origin['CI_Gender'].value_counts()

Female    50383
Male      25374
Name: CI_Gender, dtype: int64

In [20]:
# Rename variables
origin = origin.rename(columns={'AEC_Event Start Date': 'Start_Date', 'PRI_Product Role': 'Role', 
                                'PRI_Reported Brand/Product Name': 'Brand/Product', 'PRI_FDA Industry Name': 'Industry'})                            
origin = origin.rename(columns={'CI_Age at Adverse Event': 'Age', 'CI_Age Unit': 'Age', 'CI_Age Unit': 'Age Unit', 
                                'CI_Gender': 'Gender'})

In [21]:
#origin['PRI_FDA Industry Name'] = origin['PRI_FDA Industry Name'].apply(lambda x: x.split('/')[0])

In [22]:
#origin['PRI_FDA Industry Name']

In [23]:
#origin['PRI_FDA Industry Name'][(origin['CI_Gender'] == 'Female') & (origin['CI_Age Unit'] == 'Year(s)')].value_counts()

In [24]:
len(origin.index)

80169

In [36]:
clean_data = origin
clean_data.to_csv('clean_data.txt')

In [33]:
print origin.dtypes

RA_Report #                    object
RA_CAERS Created Date          object
Start_Date                     object
Role                           object
Brand/Product                  object
PRI_FDA Industry Code          object
Industry                       object
Age                           float64
Age Unit                       object
Gender                         object
AEC_One Row Outcomes           object
SYM_One Row Coded Symptoms     object
Age_interval                   object
dtype: object


In [32]:
origin['Age_interval'] = origin['Age_interval'].astype('str')

In [30]:
origin['Age_interval'][origin['Age Unit'] == 'Month(s)'].value_counts()

0-3     371
3-6     282
6-9     233
9-12    111
Name: Age_interval, dtype: int64